<a href="https://colab.research.google.com/github/ElioAlmiron/Starting_repository/blob/main/Opera%C3%A7%C3%B5es_na_Clear_Corretora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Extração de dados para declaração de imposto de renda de Notas de Corretagem - Clear Corretora**

Todo início de ano é necessário fazer a declaração de imposto de renda. E é uma das atividades mais chatas desse período - pior que pegar congestionamento na volta do Carvanal, que nos toma um bom tempo e nos gera certo stress. Sempre há o risco de fazer a declaração incorretamente e, como consequência, cair na malha fina da Receita Federal. E para quem opera investe em ações, há um ponto adicional: inserir as operações realizadas no ano fiscal. Para isso, é recomendado planilhar cada uma dessas transações para facilitar a declaração. 


Desse modo, montei um programa em Python que lê cada uma das Notas de Negociação e exporta cada uma das informações relevantes para declaração para um arquivo em Excel. Fiz isso tanto para ganhar tempo quanto para ter maior precisão nos dados, uma vez que a tarefa passa a ser automatizada, gerando segurança na informação passada à Receita Federal. 


Observação: utilizei como base a Nota de Corretagem da Clear do Corretora, do Grupo XP, mas o código pode ser adaptado para qualquer outra Nota de Negociação, uma vez que a lógica não é alterada.

**É importante ressaltar que não sou contador, mas sim engenheiro, e fiz esse material apenas para facilitar a minha vida e o disponibilizo a título de conhecimento. Caso desejem utilizá-lo, peço que não o considerem para as suas declarações sem antes validar cada um dos registros com todas as Notas de Negociação.**

##**Bibliotecas utilizadas:**



1.   Utilizei a biblioteca ***OS*** para interagir com o sistema operacional através do Python, acessando a pasta onde as Notas de Corretagem se encontram.

2.   Utilizei a biblioteca **PDF Plubber** para acessar os arquivos PDF pelo Python.

3.   Utilizei a biblioteca ***pandas*** para exportar o *output* para o Excel, que facilita caso haja interesse em enviar o arquivo a um contador.

##**Acessar as Notas de Negociação**

Primeiramente, é necessário salvar todas as Nota de Negociação em uma pasta, que será acessada pelo Python.


O comando ***os.chdir*** nos permite inserir o caminho onde essa pasta se encontra:

In [8]:
#importar OS
import os

In [3]:
os.chdir('/content/Notas_de_negociação')

Na sequência, é criada a lista ***Notas*** com cada um dos arquivos referentes às Notas de Negociação:

In [4]:
Notas = os.listdir()
Notas

['NotaNegociacao_418119_20200325.pdf']

E então a variável ***Quant_Notas*** salva a quantidade de Notas de Negociação. Isso facilita o acesso a cada nota, pois a leitura de todas será automatizada por meio de uma estrutura de repetição até que a última nota seja processada.

In [5]:
Quant_Notas = len(Notas)

##**Ler cada Nota de Negociação e salvar todos os registros em uma lista**

Primeiro deverá ser criar uma lista que servirá como base para o registro de todas as operações:

In [ ]:
base_atual = []

E por fim, com uma estrutura de repetição, cada Nota de Negociação será tratada, havendo o registro das seguintes informações:

*   Data da operação
*   Operação de Compra ou Venda
*   Tipo de Mercado
*   Empresa
*   Especificação da ação (ON, PN, Unit)
*   Quantidade
*   Preço da ação
*   Valor da Operação
*   Operação de Débito ou Crédito












In [10]:
# Instalar o PDPF Plumber
!pip install pdfplumber

# importar o PDF Plumber
import pdfplumber

     |████████████████████████████████| 51kB 2.9MB/s 
     |████████████████████████████████| 5.6MB 6.1MB/s 
     |████████████████████████████████| 143kB 18.9MB/s 
     |████████████████████████████████| 1.9MB 28.0MB/s 
  Created wheel for pdfplumber: filename=pdfplumber-0.5.26-cp37-none-any.whl size=31747 sha256=378835b3eb53477f718500803649e7acad0ad116e6e8d89a74efee1a71f0ff00
  Stored in directory: /root/.cache/pip/wheels/5a/4a/09/8f7b594a0d7ed1ca5fe27078a6a96e7439167a502074d87d1b
Successfully built pdfplumber


In [ ]:
# abrir o arquivo
a = 0
while a < Quant_Notas:
  
  # abrir o PDF de índice "a"
  pdf = pdfplumber.open(Notas[a])
  
  # referenciar a primeira página do PDF (índice 0, onde estarão os dados)
  page = pdf.pages[0]
  
  # extrair informações da página
  text = page.extract_text()

  # transformar página em uma lista e cada linha um elemento dessa página
  text.split('\n')

  registros = [] # cada registro serão adicionados à lista 'registros'
  
  

  i = 19 # a próxima informação está na linha de índice 19
  while text.split('\n')[i][0] == "1": # será avaliada cada linha com registro desde a primeira. Quando começar com algo diferente de "1", não será registro.
    # leitura da data da operação

    Data = text.split('\n')[2][-10:]

    # adição da data na lista de registros

    registros.append(Data)
  
    # negociacao: retira o campo "Negociação" do registro e o adiciona na lista registros
    #negociacao = text.split('\n')[i][0:9].strip()
    #registros.append(negociacao)

    # C/V: retira o campo "C/V" e o adiciona na lista de registros
    CV = text.split('\n')[i][10].strip()
    registros.append(CV)

    # tipo_mercado: retira o campo "Tipo mercado" e o adiciona na lista de registros
    tipo_mercado = text.split('\n')[i][12:17]
    registros.append(tipo_mercado)

    # especificacao_do_titulo: retira o campo "Especificação do Título" e o adiciona na lista de registros
    
    j = 18
    espaco = 0
    
    while espaco < 2: # não pode chegar a dois espaços. Caso chegue, não é mais separação do nome (exemplo: "Santos Brasil" tem um só espaço)
      if text.split('\n')[i][j] == " ":
        espaco = espaco + 1
      j = j + 1
    

    nome = text.split('\n')[i][18:j-2]
    registros.append(nome)

    n = 0
    codigo = 0

    while n == 0: # "n" é o controle para sair do loop 
      if text.split('\n')[i][j] == " ":
        j = j + 1
      else:
        codigo = text.split('\n')[i][j:j+6]
        n = 1
    
    registros.append(codigo)
    
    # Quantidade: retira o campo "Quantidade" e o adiciona na lista de registros
    
    if codigo == "UNT N2":
      j = j + 7 # inicia a partir do primeiro algarismo da quantidade
      k = j
      espaco = 0 # não pode ter um espaco, pois é número inteiro (quantidade). Quando encontrar o espaço, a quantidade está definida
    else:
      j = j + 6 # inicia a partir do primeiro algarismo da quantidade
      k = j
      espaco = 0 # não pode ter um espaco, pois é número inteiro (quantidade). Quando encontrar o espaço, a quantidade está definida
    
    while espaco == 0:
      if text.split('\n')[i][j] == " ":
        espaco = espaco + 1
      j = j + 1
    quantidade = text.split('\n')[i][k:j-1]

    registros.append(quantidade)
    
    # preco_ajuste: retira o campo "preco_ajuste" e o adiciona na lista de registros

    k = j
    espaco = 0

    while espaco == 0:
      if text.split('\n')[i][j] == " ":
        espaco = espaco + 1
      j = j + 1
    preco_ajuste = text.split('\n')[i][k:j-1]

    registros.append(preco_ajuste)


    # valor: retira o campo "valor" e o adiciona na lista de registros

    k = j
    espaco = 0

    while espaco == 0:
      if text.split('\n')[i][j] == " ":
        espaco = espaco + 1
      j = j + 1
    valor = text.split('\n')[i][k:j-1]

    registros.append(valor)

    # DC: retira o campo "D/C" e o adiciona na lista de registros

    DC = text.split('\n')[i][-1]

    registros.append(DC)
    
    # imprime o registro
    print('Registro', i- 18, 'arquivo' , a + 1 , ':', registros)

    base_atual.append(registros)

    registros = [] 
    
    i = i + 1

  a = a + 1



Registro 1 arquivo 1 : ['25/03/2020', 'C', 'VISTA', 'PETROBRAS', 'PN N2 ', '500', '13,32', '6.660,00', 'D']
Registro 2 arquivo 1 : ['25/03/2020', 'C', 'VISTA', 'TAESA', 'PN N2 ', '200', '8,12', '1.624,00', 'D']
Registro 3 arquivo 1 : ['25/03/2020', 'C', 'VISTA', 'TAESA', 'PN N2 ', '100', '8,12', '812,00', 'D']
Registro 1 arquivo 2 : ['14/12/2020', 'C', 'VISTA', 'ALUPAR', 'UNT N2', '100', '26,81', '2.681,00', 'D']
Registro 2 arquivo 2 : ['14/12/2020', 'C', 'VISTA', 'HYPERA', 'ON NM ', '100', '32,60', '3.260,00', 'D']
Registro 3 arquivo 2 : ['14/12/2020', 'C', 'VISTA', 'ODONTOPREV', 'ON NM ', '100', '13,90', '1.390,00', 'D']
Registro 4 arquivo 2 : ['14/12/2020', 'V', 'VISTA', 'PETROBRAS', 'PN N2 ', '300', '28,07', '8.421,00', 'C']
Registro 5 arquivo 2 : ['14/12/2020', 'C', 'VISTA', 'WIZ S.A', 'ON NM ', '200', '9,31', '1.862,00', 'D']


A base se encontrará abaixo:

In [ ]:
#print('Base atual:' , base_atual)
print('Base atual:' , base_atual)

Base atual: [['25/03/2020', 'C', 'VISTA', 'PETROBRAS', 'PN N2 ', '500', '13,32', '6.660,00', 'D'], ['25/03/2020', 'C', 'VISTA', 'TAESA', 'PN N2 ', '200', '8,12', '1.624,00', 'D'], ['25/03/2020', 'C', 'VISTA', 'TAESA', 'PN N2 ', '100', '8,12', '812,00', 'D'], ['14/12/2020', 'C', 'VISTA', 'ALUPAR', 'UNT N2', '100', '26,81', '2.681,00', 'D'], ['14/12/2020', 'C', 'VISTA', 'HYPERA', 'ON NM ', '100', '32,60', '3.260,00', 'D'], ['14/12/2020', 'C', 'VISTA', 'ODONTOPREV', 'ON NM ', '100', '13,90', '1.390,00', 'D'], ['14/12/2020', 'V', 'VISTA', 'PETROBRAS', 'PN N2 ', '300', '28,07', '8.421,00', 'C'], ['14/12/2020', 'C', 'VISTA', 'WIZ S.A', 'ON NM ', '200', '9,31', '1.862,00', 'D']]


##**Exportar a base para o Excel**

A base será exportada via pandas para o arquivo "Operações realizadas". Desse modo, o arquivo estará pronto para **validação** entre cada um de seus registros e as Notas de Corretagem.

In [ ]:
import pandas as pd

df = pd.DataFrame(base_atual)
df.to_excel(excel_writer = "Operações realizadas.xlsx")